# Dependencies

## Ensure correct version of python
[Set ipython kernel](https://stackoverflow.com/questions/60775160/install-python-3-8-kernel-in-google-colaboratory/71511943#71511943)

In [1]:
!wget -O mini.sh https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
!chmod +x mini.sh
!bash ./mini.sh -b -f -p /usr/local
!conda install -q -y jupyter
!conda install -q -y google-colab -c conda-forge
!python -m ipykernel install --name "py38" --user

--2023-09-28 12:13:10--  https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89817099 (86M) [application/x-sh]
Saving to: ‘mini.sh’

mini.sh             100%[===================>]  85.66M   203MB/s    in 0.4s    

2023-09-28 12:13:10 (203 MB/s) - ‘mini.sh’ saved [89817099/89817099]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py38_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py38_0
    - cffi==1.14.0=py38h2e261b9_0
    - chardet==3.0.4=py38_1003
    - conda-package-handling==1.6.0=py38h7b6447c_0
    - conda==4.8.2=py38_0
    - cryptography==2.8=

In [1]:
import sys
print(sys.version)

3.8.18 (default, Sep 11 2023, 13:20:55) 
[GCC 11.2.0]


## Tensorflow lite model maker requirements

In [2]:
!wget https://raw.githubusercontent.com/tensorflow/examples/master/tensorflow_examples/lite/model_maker/requirements.txt

--2023-09-28 12:18:36--  https://raw.githubusercontent.com/tensorflow/examples/master/tensorflow_examples/lite/model_maker/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 812 [text/plain]
Saving to: ‘requirements.txt.1’

requirements.txt.1  100%[===================>]     812  --.-KB/s    in 0s      

2023-09-28 12:18:36 (63.6 MB/s) - ‘requirements.txt.1’ saved [812/812]



### Installation

In [47]:
!pip uninstall -y tensorflow && pip install tensorflow==2.8.0
!pip install pycocotools
!pip install protobuf==3.20
!pip install -r requirements.txt
!pip install tflite_model_maker

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0
     |████████████████████████████████| 454.4 MB 31 kB/s 
     |████████████████████████████████| 14.9 MB 67.0 MB/s 
     |████████████████████████████████| 462 kB 64.8 MB/s 
     |████████████████████████████████| 132 kB 87.4 MB/s 
     |████████████████████████████████| 1.2 MB 62.5 MB/s 
     |████████████████████████████████| 4.0 MB 77.8 MB/s 
     |████████████████████████████████| 64 kB 3.4 MB/s 
     |████████████████████████████████| 4.4 MB 67.5 MB/s 
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=af8ebd0a7eb8610fbf0fd08423582ca21a3672b6ea502cb5bd84daef76cd8050
  Stored in directory: /root/.cache/pip/wheels/a0/16/9c/5473df82468f958445479c59e784896fa24f4a5fc024b0f501
  Created wheel for wrapt: filename=wrapt-1.12.1-cp38-cp38-linux_x86_64.whl size=75812 sha256=8e71de6b5c9f73a0ee593db3b42d957a6ffe80da233209917c5fdd2d4924

# Import dependencies

In [48]:
import numpy as np
import os
import shutil
import random
import pandas as pd
import ast

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf


# Load dataset

In [7]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
  inflating: dataset/.DS_Store       
  inflating: __MACOSX/dataset/._.DS_Store  
   creating: dataset/images/
  inflating: dataset/balloon-data.csv  
  inflating: dataset/images/605521662_a470fef77f_b.jpg  
  inflating: dataset/images/9330497995_4cf0438cb6_k.jpg  
  inflating: dataset/images/5178670692_63a4365c9c_b.jpg  
  inflating: dataset/images/25899693952_7c8b8b9edc_k.jpg  
  inflating: dataset/images/14321263043_b76ef054d3_k.jpg  
  inflating: dataset/images/1297451346_5b92bdac08_b.jpg  
  inflating: dataset/images/126700562_8e27720147_b.jpg  
  inflating: dataset/images/2311771643_f46392fcc0_b.jpg  
  inflating: dataset/images/4838031651_3e7b5ea5c7_b.jpg  
  inflating: dataset/images/518678836_94d58e3839_b.jpg  
  inflating: dataset/images/5560377994_cb597a4af5_b.jpg  
  inflating: dataset/images/3927754171_9011487133_b.jpg  
  inflating: dataset/images/9210739293_2b0e0d991e_k.jpg  
  inflating: dataset/images/15717689633_5f7f78c28e_k

In [8]:
!rm -rf dataset/train >2 ; mkdir dataset/train
!rm -rf dataset/validation >2 ; mkdir dataset/validation
!rm -rf dataset/test >2 ; mkdir dataset/test

## Spilt datset

In [9]:

classes = ['Balloon']

images = os.listdir('dataset/images')
number_of_images = len(images)

train_percent = 0.8
validation_percent = 0.1
test_percent = 0.1

# Python will always round down, test_number_images gets the remaining images
train_number_of_images = int(number_of_images * train_percent)
validation_number_of_images = int(number_of_images * validation_percent)
test_number_of_images = number_of_images - train_number_of_images - validation_number_of_images

def relative_coordinate(maximum: int, value: int):
   return np.float32(value/maximum)[0]

def move_files(amount: int, label: str, images: list):
  path = rf'dataset/{label}'
  all_meta = pd.read_csv(rf'dataset/balloon-data.csv')
  all_meta['bbox'] = all_meta['bbox'].apply(ast.literal_eval)

  print(f'Moving {amount} files to {path}')
  meta = pd.DataFrame(columns=['label', 'path', 'class', 'x_min', 'y_min','a','b',
                              'x_max', 'y_max', 'c', 'd'])
  for i in range(amount):
      file = random.choice(images)
      print(rf'dataset/images/{file}')
      print(rf'{path}/{file}')
      entry = all_meta.loc[all_meta['fname'] == file]
      for boxes in entry['bbox']:
        for box in boxes:
          meta.loc[len(meta)] = {'label': label.upper(), 'path': rf'{path}/{file}', 'class': 'Balloon',
                       'x_min': relative_coordinate(entry['width'] ,box['xmin']),
                       'y_min': relative_coordinate(entry['height'], box['ymin']),
                       'x_max': relative_coordinate(entry['width'], box['xmax']),
                       'y_max': relative_coordinate(entry['height'], box['ymax'])}
      shutil.copy(rf'dataset/images/{file}', rf'{path}/{file}')
      images.remove(file)
  return meta, images

train_meta_data, images = move_files(train_number_of_images, 'train', images)
validation_meta, images = move_files(validation_number_of_images, 'validation', images)
test_meta, images = move_files(test_number_of_images, 'test', images)

meta = pd.concat([train_meta_data, validation_meta, test_meta])

meta.to_csv('dataset-meta.csv', index=False, header=False)

Moving 59 files to dataset/train
dataset/images/4838031651_3e7b5ea5c7_b.jpg
dataset/train/4838031651_3e7b5ea5c7_b.jpg
dataset/images/154446334_5d41cd1375_b.jpg
dataset/train/154446334_5d41cd1375_b.jpg
dataset/images/8436015314_3a678c1005_k.jpg
dataset/train/8436015314_3a678c1005_k.jpg
dataset/images/120853323_d4788431b9_b.jpg
dataset/train/120853323_d4788431b9_b.jpg
dataset/images/3945575930_ce99a7e98d_b.jpg
dataset/train/3945575930_ce99a7e98d_b.jpg
dataset/images/126700562_8e27720147_b.jpg
dataset/train/126700562_8e27720147_b.jpg
dataset/images/6810773040_3d81036d05_k.jpg
dataset/train/6810773040_3d81036d05_k.jpg
dataset/images/9330497995_4cf0438cb6_k.jpg
dataset/train/9330497995_4cf0438cb6_k.jpg
dataset/images/12288355124_5e340d3de3_k.jpg
dataset/train/12288355124_5e340d3de3_k.jpg
dataset/images/145053828_e0e748717c_b.jpg
dataset/train/145053828_e0e748717c_b.jpg
dataset/images/518678836_94d58e3839_b.jpg
dataset/train/518678836_94d58e3839_b.jpg
dataset/images/24362039530_b151b41a52_k.

In [ ]:
!wget https://tfhub.dev/google/qat/mobilenet_v2_retinanet_256/1

# Train model

In [49]:
model_template = 'efficientdet_lite2'
model_name = 'turret-syndrome'
batch_size = 7
epochs=100

train, validation, test = object_detector.DataLoader.from_csv('dataset-meta.csv')
model = object_detector.create(train, model_spec=model_spec.get(model_template),
                             batch_size=batch_size, train_whole_model=True,
                             validation_data=validation, epochs=epochs)



Epoch 1/100
6/8 [=====================>........] - ETA: 0s - det_loss: 1.8375 - cls_loss: 1.1376 - box_loss: 0.0140 - reg_l2_loss: 0.0759 - loss: 1.9134 - learning_rate: 0.0082 - gradient_norm: 1.1131

8/8 [==============================] - 70s 3s/step - det_loss: 1.8027 - cls_loss: 1.1299 - box_loss: 0.0135 - reg_l2_loss: 0.0759 - loss: 1.8786 - learning_rate: 0.0084 - gradient_norm: 1.0723 - val_det_loss: 1.6671 - val_cls_loss: 1.0973 - val_box_loss: 0.0114 - val_reg_l2_loss: 0.0759 - val_loss: 1.7430
Epoch 2/100
8/8 [==============================] - 8s 1s/step - det_loss: 1.6338 - cls_loss: 1.0625 - box_loss: 0.0114 - reg_l2_loss: 0.0759 - loss: 1.7097 - learning_rate: 0.0087 - gradient_norm: 1.2378 - val_det_loss: 1.4130 - val_cls_loss: 0.9523 - val_box_loss: 0.0092 - val_reg_l2_loss: 0.0759 - val_loss: 1.4888
Epoch 3/100
8/8 [==============================] - 12s 2s/step - det_loss: 1.3732 - cls_loss: 0.8780 - box_loss: 0.0099 - reg_l2_loss: 0.0759 - loss: 1.4491 - learning_rate: 0.0087 - gradient_norm: 1.9072 - val_det_loss: 1.0875 - val_cls_loss: 0.7085 - val_box_loss: 0.0076 - val_reg_l2_loss: 0.0759 - val_loss: 1.1633
Epoch 4/100
8/8 [==============================] - 8s 1s

# Evaluate result

In [51]:
eval = model.evaluate(test)
#print(f'Loss = {loss}, Accuracy = {accuracy}')
model_name = f'{model_name}-{model_template}'
model.export(export_dir=model_name, tflite_filename=f'f16-{model_name}.tflite',
            quantization_config=QuantizationConfig.for_float16())
model.export(export_dir=model_name, tflite_filename=f'i8-{model_name}.tflite',
            quantization_config=QuantizationConfig.for_int8((train)))
model.export(export_dir=model_name, tflite_filename=f'{model_name}.tflite')
model_name = ''

1/1 [==============================] - 8s 8s/step



2023-09-28 15:04:32.677765: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.
2023-09-28 15:04:43.608730: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-09-28 15:04:43.608785: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-09-28 15:04:43.622639: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpnjg0np47
2023-09-28 15:04:43.920139: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-09-28 15:04:43.920208: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpnjg0np47
2023-09-28 15:04:44.851417: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2023-09-28 15:04:50.081598: I tensorflow/cc/saved_